In [10]:
import torch
import pytorch_lightning as pl
from pytorch_forecasting import TemporalFusionTransformer
import torch.nn as nn

# Define the model class as provided
class TemporalFusionTransformerModule(pl.LightningModule):
    def __init__(self, model: TemporalFusionTransformer):
        super(TemporalFusionTransformerModule, self).__init__()
        self.model = model

    def compute_loss(self, y_hat, y):
        losses = []
        for feature_idx in range(y_hat.shape[2]):
            feature_pred = y_hat[:, :, feature_idx]
            feature_loss = nn.MSELoss()(feature_pred, y)
            losses.append(feature_loss)

        total_loss = torch.mean(torch.stack(losses))
        return total_loss

    def forward(self, x):
        out = self.model(x)
        return out[0]

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.compute_loss(y_hat, y[0])
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        val_loss = self.compute_loss(y_hat, y[0])
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
            }
        }

    def predict_step(self, batch, batch_idx):
        if isinstance(batch, tuple):
            x = batch[0]
        else:
            x = batch
        return self(x)

# Instantiate the TemporalFusionTransformer model
# You should use the same hyperparameters that were used during training
temporal_fusion_transformer = TemporalFusionTransformer(
    hidden_size=32
)

# Instantiate your LightningModule with the TemporalFusionTransformer model
loaded_model = TemporalFusionTransformerModule(model=temporal_fusion_transformer)

# Load the saved weights
loaded_model.load_state_dict(torch.load('trained_model_weights.pth'))

# Set the model to evaluation mode
loaded_model.eval()


C:\Users\Mazen\AppData\Local\Temp\ipykernel_26148\332089907.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('trained_model_weigh

RuntimeError: Error(s) in loading state_dict for TemporalFusionTransformerModule:
	Missing key(s) in state_dict: "model.static_context_variable_selection.fc1.weight", "model.static_context_variable_selection.fc1.bias", "model.static_context_variable_selection.fc2.weight", "model.static_context_variable_selection.fc2.bias", "model.static_context_variable_selection.gate_norm.glu.fc.weight", "model.static_context_variable_selection.gate_norm.glu.fc.bias", "model.static_context_variable_selection.gate_norm.add_norm.norm.weight", "model.static_context_variable_selection.gate_norm.add_norm.norm.bias", "model.static_context_initial_hidden_lstm.fc1.weight", "model.static_context_initial_hidden_lstm.fc1.bias", "model.static_context_initial_hidden_lstm.fc2.weight", "model.static_context_initial_hidden_lstm.fc2.bias", "model.static_context_initial_hidden_lstm.gate_norm.glu.fc.weight", "model.static_context_initial_hidden_lstm.gate_norm.glu.fc.bias", "model.static_context_initial_hidden_lstm.gate_